# Lab Two: Classification

We will build upon the predictive analysis (classification) that we completed in the
previous mini-project, adding additional modeling from new classification algorithms as well as
more explanations that are inline with the CRISP-DM framework.

We chose to continue to use the CIFAR-10 dataset. We identified the two tasks from the dataset to classify. 
Task 1: Cats and dogs
Task 2: Birds and airplanes


By Martin Garcia, Joanna Duran, Daniel Byrne

## Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
import time
import sklearn.naive_bayes as b
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import SGDClassifier
from sklearn.utils import compute_class_weight

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Summary of Minilab

### Business Understanding
Our objective was to create a logistic regression model and a support vector machine model for the classification of each image. We were to determine which model is best suited for this standard classification task based on a comparison on their prediction accuracy, training times, and computational efficiency.

The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly used to train machine learning and computer vision algorithms. It is one of the most widely used datasets for machine learning research. It is a subset of the 80 million tiny images dataset and consists of 60,000 32x32 color images containing one of 10 object classes, with 6,000 images per class.

### Data Meaning Type

#### The CIFAR-10 dataset
We are using the [CIFAR-10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset which consists of 60,000 32x32 colour images in 10 classes, with 6,000 images per class. The dataset includes are 50,000(80%) training images and 10,000(20%) test images broken in to 5 pre-randomized training batches and 1 test batch.

Each training batch contains 10,000 observations with a row vector of length 3,072 representative of color image of 32x32 pixels. The first 1,024 columns consist of red values, followed by green, and blue. The data also incorporates labels ranging from 0 to 9 and are listed below.

* airplane : 0
* automobile : 1
* bird : 2
* cat : 3
* deer : 4
* dog : 5
* frog : 6
* horse : 7
* ship : 8
* truck : 9

The test batch contains 1,000 randomly-selected images from each class. The 5 training batches are randomized and contain a variable number of images from each class. 

The classes are completely mutually exclusive. There is no overlap between automobiles and trucks. "Automobile" includes sedans, SUVs, things of that sort. "Truck" includes only big trucks. Neither includes pickup trucks.

#### Simple Statistics

In [ ]:
#load the data, reshape to 32x32 matrix per color, transpose matrices
def load_cfar10_batch(path, batch_id = None, reshape = True):
    """
    Argument:
    path -- path the datasets
    batch_id -- id of the batch (1 to 5) to load
    
    Returns:
    parameters -- python dictionary containing:
                    X -- features
                    Y -- labels
    """
    if batch_id is not None:
        filepath = path + 'data_batch_' + str(batch_id)
    else:
        filepath = path
        
    with open(filepath, mode='rb') as file:
        # note the encoding type is 'latin1'
        batch = pickle.load(file, encoding='latin1')
    
    if reshape:    
        X = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
        Y = np.array(batch['labels'])
    else:
        X = batch['data']
        Y = np.array(batch['labels'])
        
    return X, Y

In [ ]:
def load_cfar10_dataset():
    """
    Loads the cfar10 dataset
    
    Arguments: 
        None
    
    Returns:
        X - Training dataset
        Y - Training labels
         - Test datset
        y_test - test labelss
    """
    x_test,y_test = load_cfar10_batch("data/test_batch",None,False)
    X,Y = load_cfar10_batch("data/",1,False)

    for n in range(2,6):
        x,y = load_cfar10_batch("data/",n,False)    
        X = np.concatenate((X,x),axis=0)
        Y = np.concatenate((Y,y),axis=0)


    return (X,Y,x_test,y_test)

In [ ]:
#load labels for our label
def load_label_names():
    return ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

#display images
def display_stats(data, batch_id, sample_id):
    features, labels = load_cfar10_batch(data, batch_id)

    if not (0 <= sample_id < len(features)):
        print('{} samples in batch {}.  {} is out of range.'.format(len(features), batch_id, sample_id))
        return None

    print('\nStats of batch #{}:'.format(batch_id))
    print('# of Samples: {}\n'.format(len(features)))

    label_names = load_label_names()
    label_counts = dict(zip(*np.unique(labels, return_counts=True)))
    for key, value in label_counts.items():
        print('Label Counts of [{}]({}) : {}'.format(key, label_names[key].upper(), value))

    sample_image = features[sample_id]
    sample_label = labels[sample_id]

    print('\nExample of Image {}:'.format(sample_id))
    print('Image - Min Value: {} Max Value: {}'.format(sample_image.min(), sample_image.max()))
    print('Image - Shape: {}'.format(sample_image.shape))
    print('Label - Label Id: {} Name: {}'.format(sample_label, label_names[sample_label]))
    plt.axis('off')
    plt.imshow(sample_image)

batch_id = random.randint(1,5)
sample_id = random.randint(1,10000)
display_stats( "data/", batch_id, sample_id)

## Data Preparation Part 1

In this section we will define and prepare our class variables. 

Each of the batch files contains a dictionary with the following elements: 

data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image. 

labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

The dataset contains another file, called batches.meta. It too contains a Python dictionary object. It has the following entries: 

label_names -- a 10-element list which gives meaningful names to the numeric labels in the labels array described above. For example, label_names[0] == "airplane", label_names[1] == "automobile", etc.

"Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis."

In [ ]:
def simplify_labels(df,indexes,val = 10):
    """
    Return a modified label dataset filtered ot use only the classes specified

    Arguments:
    indees: indexes to filter
    df : label dataset
    val: new label value

    Returns:
    fdf - filtered df
      
    """

    def f(n):
        if n in indexes:
            return n
        else:
            return val
    
    return list(map(f,df))


In [ ]:
#load the data
X,Y,x_test,y_test = load_cfar10_dataset()

# Simplify datasets to the classes we care about
Ycatdog = simplify_labels(Y,[3,5],10)
y_test_catdog =simplify_labels(y_test,[3,5],10)
Ybirdplane = simplify_labels(Y,[0,2],10)
y_test_birdplane = simplify_labels(y_test,[0,2],10)

## Data Preparation Part 2

The final training dataset is identical to the original.  However, we've created 4 new label datasets

- Ycatdog, y_test_catdog  : Cat and dog labels are retained, all others forced to next unused label
- Ybirdplane,y_test_birdplane : Bird and plane  labels are retained, all others forced to next unused label




## Modeling and Evaluation 1

"Choose and explain your evaluation metrics that you will use (i.e., accuracy,
precision, recall, F-measure, or any metric we have discussed). Why are the measure(s) appropriate for analyzing the results of your modeling? Give a detailed explanation backing up any assertions."

## Modeling and Evaluation 2

 We will assess how well each model performs (use 80/20 training/testing split)and adjust parameters of the models to make them more accurate. 


"Choose the method you will use for dividing your data into training and
testing splits (i.e., are you using Stratified 10-fold cross validation? Why?). Explain why
your chosen method is appropriate or use more than one method as appropriate. For example, if you are using time series data then you should be using continuous training and testing sets across time."

## Modeling and Evaluation 3

We identified the two tasks from the dataset to classify. For each task we will create three different classification/regression models

Task 1: Cats and dogs

    Model A: Naive Bayes
    Model B: KNN
    Model C: Logistic regression
    
Task 2: Birds and airplanes

    Model A: Naive Bayes
    Model B: Decision Tree
    Model C: SVM

"Create three different classiﬁcation/regression models (e.g., random forest, KNN, and SVM). Two modeling techniques must be new (but the third could be SVM or logistic regression). Adjust parameters as appropriate to increase generalization performance using your chosen metric"


### Task1 Model A
(Cats and Dogs- Naive Bayes)
    
We chose several implementations of the Naive Bayes classifier: GaussianNB, MultinomialNB, and BernoulliNB.


In [ ]:
def buildmodel(nmodel,X,Y,x_test,y_test,**modelargs):
    """
    Builds one of a subset of sklearn models and returns a score
    
    Arguments:
    modeltype -- One of either "multi", "gauss", "bernoulli", "c", "nn" 
    
    """



    if nmodel == 'multi':
        m = b.MultinomialNB(**modelargs)
    elif nmodel == 'gauss':
        m = b.GaussianNB(**modelargs)
    elif nmodel == 'bernoulli':
        m = b.BernoulliNB(**modelargs)
    elif nmodel == "mlp":
        m = MLPClassifier(**modelargs)
    elif nmodel == "svm":
        m = SGDClassifier(**modelargs)
    elif nmodel == "BernoulliRBM":
        m = BernoulliRBM(**modelargs)
    elif nmodel == "LogisticRegression":
        m = LogisticRegression(**modelargs)
    
    m.fit(X, Y)
    score = m.score(x_test,y_test) * 100

    return m,score

In [ ]:
multiNB,multiNBScore = buildmodel('multi',X,Ycatdog,x_test,y_test_catdog, alpha = .01)
print("MultinomialNB score :%",round(multiNBScore,2))

In [ ]:
gaussNB, gaussNBScore = buildmodel('gauss',X,Ycatdog,x_test,y_test_catdog,priors=None, var_smoothing=1e-09)
print("GaussianNB score :%",round(gaussNBScore,2))

In [ ]:
berboulliNB, bernoulliNBScore = buildmodel('bernoulli',X,Ycatdog,x_test,y_test_catdog,alpha = .01)
print("BernoulliNB score :%",round(bernoulliNBScore,2))

In [ ]:
mlp, mlpScore = buildmodel('mlp',X,Ycatdog,x_test,y_test_catdog,
                           activation = 'relu',hidden_layer_sizes=(10,),learning_rate ='adaptive',verbose = True)
print("Multi-layer Perceptron score :%",round(mlpScore,2))

In [ ]:
lr,lrScore = buildmodel('LogisticRegression',X,Ycatdog,x_test,y_test_catdog,
                       solver='newton-cg', tol=1,multi_class='multinomial', verbose = True)
print("LogisticRegression score :%",round(lrScore,2))

### Task1 Model B
(Cats and Dogs- KNN)

### Task1 Model C
(Cats and Dogs- Logistic Regression)

### Task2 Model A
(Birds and Airplanes- Naive Bayes)

### Task2 Model B
(Birds and Airplanes- Decision Tree)

### Task2 Model C
(Birds and Airplanes- SVM)

## Modeling and Evaluation 4

"Analyze the results using your chosen method of evaluation. Use visualizations of the results to bolster the analysis. Explain any visuals and analyze why they are interesting to someone that might use this model."

## Modeling and Evaluation 5

Advantages of Task 1: Cats and dogs Model A: Random Forest

    *
Advantages of Task 1: Cats and dogsModel B: KNN

    *
Advantages of Task 1: Cats and dogsModel C: Logistic regression

    *
Advantages of Task 2: Birds and airplanes Model A: Naive Bayes

    *
Advantages of Task 2: Birds and airplanes Model B: Decision Tree

    *
Advantages of Task 2: Birds and airplanes Model C: SVM

    *

"Discuss the advantages of each model for each classification task, if any. If there are not advantages, explain why. Is any model better than another? Is the difference significant with 95% confidence? Use proper statistical comparison methods. You must use statistical comparison techniques—be sure they are appropriate for your chosen method of validation as discussed in unit 7 of the course."

## Modeling and Evaluation 6

 Which attributes from your analysis are most important? Use proper methods discussed in class to evaluate the importance of different attributes. Discuss the results and hypothesize about why certain attributes are more important than others for a given classiﬁcation task. Attributes are pixel representative of colors red green and blue  

In [ ]:
np.min(X),np.max(X)

In [ ]:

scaler = StandardScaler()
# Fit on training set only.
scaler.fit(X)
# Apply transform to both the training set and the test set.
X_trans = scaler.transform(X)

In [ ]:
np.min(X_trans),np.max(X_trans)

In [ ]:
X_trans.shape


In [ ]:
pca_cifar = PCA(n_components=5)
principalComponents_cifar = pca_cifar.fit_transform(X)

In [ ]:
principal_cifar_Df = pd.DataFrame(data = principalComponents_cifar
             , columns = ['prin 1', 'prin 2','prin 3','prin 4','prin 5'])
principal_cifar_Df['y'] = Y

In [ ]:
principal_cifar_Df.str

In [ ]:
print('Explained variation per principal component: {}'.format(pca_cifar.explained_variance_ratio_))

## Deployment

We believe that our model would be best suited for

"How useful is your model for interested parties (i.e., the companies or organizations that might want to use it for prediction)? How would you measure the model's value if it was used by these parties? How would your deploy your model for interested parties? What other data should be collected? How often would the model need to be updated, etc.?"

#### OLD- MAY DELETE -Create Models

In this section we will create a logistic regression model and a support vector machine model for the classification task involved with your dataset.  We will assess how well each model performs (use 80/20 training/testing split)and adjust parameters of the models to make them more accurate. 

One of our primary concerns was that single level logistic regression cannot handle a multiclass problem very well.  In order to tackle this issue we decided to change to simplify the problem to a binary classification exercise. We modified the training labels to be either the "CAT"/"NOT CAT" which simplified the task for our models.

#### OLD- MAY DELETE -Logistic Regression Model

In [ ]:
# Build Logistic Regression Model with Stochastic Gradient Descent 
X,Y = load_cfar10_batch("data/",1,False)

for n in range(2,6):
    x,y = load_cfar10_batch("data/",n,False)    
    X = np.concatenate((X,x),axis=0)
    Y = np.concatenate((Y,y),axis=0)

test_X,test_Y = load_cfar10_batch("data/test_batch",None,False)

# Modify the dataset labels to lable cats/not/cats
#(if_test_is_false, if_test_is_true)[test]
catY = (Y == 3)
cat_test_Y = (test_Y == 3) 

sgdlr = SGDClassifier(alpha=0.001, max_iter=5000, tol=1e-3,verbose=1,n_jobs=4,loss="log")

print("Data shape: ",X.shape)
print("Labels shape: ",catY.shape)

print("Test Data shape: ",test_X.shape)
print("Test Labels shape: ",cat_test_Y.shape)


In [ ]:
import time

start = time.time()
sgdlr.fit(X,catY)
end = time.time()
elapsed = end - start
print("Training complete.  Time elapsed = ",elapsed)

In [ ]:
sgdlrscore = sgdlr.score(test_X,cat_test_Y)
print("Logistic Regression Accuracy: ",sgdlrscore)

#### OLD- MAY DELETE -Support Vector Machine Model

In [ ]:
# Build SVM Regression Model with Stochastic Gradient Descent 
sgd_svm = SGDClassifier(alpha=0.001, max_iter=5000, verbose = 1, tol=1e-3,n_jobs=4)

In [ ]:
start = time.time()
sgd_svm.fit(X,catY)
end = time.time()

svm_elapsed = end - start
print("Training complete.  Time elapsed = ",svm_elapsed)

In [ ]:
sgd_svmscore = sgd_svm.score(test_X,cat_test_Y)
print("Support Vector Machine Model: ",sgd_svmscore)

In [ ]:
#Weights
weights = sgd_svm.coef_
weights

#### OLD- MAY DELETE -Model Advantages

We will discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency?

In [ ]:

def print_mislabeled_images(X, y, p):
    """
    Plots images where predictions and truth were different.
    X -- dataset
    y -- true labels
    p -- predictions
    """
    a = y == p
    mislabeled_indices = np.asarray(np.where(a == False))
    plt.rcParams['figure.figsize'] = (10.0, 10.0) # set default size of plots
    num_images = len(mislabeled_indices[0])
    
    print("There were "+str(num_images)+" mislabeled images.")
    print("Here is a sampling of 6 of the mislabeled images.")
    
    rand_ids = np.random.randint(num_images, size=6)
    for i in range(6):
        index = mislabeled_indices[0][i]
        plt.subplot(3, 2, i + 1)
        plt.imshow(X[index].reshape(3, 32, 32).transpose(1,2,0), interpolation='nearest')
        plt.axis('off')
        plt.title("Prediction:"+str(p[index]) + "\nLabel:"+str(y[index]))



####  OLD- MAY DELETE -Logistic Regression Mislabeled Images

In [ ]:
sgdpred_test = sgdlr.predict(test_X)
print_mislabeled_images(test_X,cat_test_Y,sgdpred_test)

#### OLD- MAY DELETE -Support Vector Machine Mislabeled Images

In [ ]:
svmpred_test = sgd_svm.predict(test_X)
print_mislabeled_images(test_X,cat_test_Y,svmpred_test)

#### OLD- MAY DELETE -Model Comparison

The two models were trained with similar hyperparameters to simplify the A/B comparison between the two models.  The only parameter that varied between the two models was the *loss* parameter. 

The loss function defaults to ‘hinge’, which gives a linear SVM. The ‘log’ loss gives logistic regression, a probabilistic classifier. 

The two models we cerated in identifying cats peformed similarly in terms of accuracy, 85% to 89% accuaracy over several test runs.

Training times varied depending on the underlying hardware and computer workload; however, the two models were comparable in overall training times on this dataset 69-72 seconds on Intel Core i7-6700 @ 3.4GHz, 24Gb of RAM, 4 Cores, and and NVIDIA 1060. 

Both models also seemed to mislabel similar images as evident in the plots above.  Both models seemed to struggle with cats in odd, off center poses, or curled up into a ball.  The one car image that was mislabaled as a cat kind of resemebles a cat with the openned hatchback of the car resembling a tail.

Overall both models performed similarily in this classification task with the chosen model parameters.

#### OLD- MAY DELETE -Interpreting Feature Importance

In this section we will use the weights from logistic regression to interpret the importance of different features for the classification task.

Individual weights represent the strength of connection between our units, in this case it is a connection from our array of pixels which come in three magnitudes of RBG colors. If the weights are overlayed as a matrix we can see the influence of specific colors to help us identify a class. The weights act as a filter coefficient for each pixel of our array of length. We define the score of each class in this case cat vs not cat based on the weighted sum of all the pixels with all images, each class sore is a linear function over this space.

In [ ]:
#Weights
print(X.shape)

weights = sgd_svm.coef_
print(weights.shape)
#plt.scatter(X,weights)

#### OLD- MAY DELETE - Interpreting Support Vectors

In this section we will look at the chosen support vectors for the classification task. 

In [ ]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X, catY) 
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)